# The Visualization Notebook for the `ga-benchmark`

This notebook presents the performance of Geometric Algebra libraries, library generators, and code optimizers included in the `ga-benchmark`. Basically, It reads data from the `.json` files produced by the `ga-benchmark` and provides pleasant tables and charts for comparing the solutions.

You need to run the entire notebook to see the results. We suggest reading the instructions contained in each text cell for a better understanding of the data.

## Inicialization

Let's import some modules.

In [7]:
import ipywidgets as widgets
from os import path, getcwd
from IPython.display import display

The `gabm` module defines the functions to read and display results produced by the `ga-benchmark` tool.

In [8]:
import sys
sys.path.append(path.abspath(getcwd()))  # Assuming that the working directory includes the 'gabm.py' file.

from gabm import *

Now, let's read the content of the `.json` files produced by the `ga-benchmark`. Set the `input_path` variable to change the directory path including the `.json` files. The `data` variable stores all data available

In [9]:
input_path = path.join(path.abspath(getcwd()), 'build')  # Assuming that the working directory is '<path-to>/ga-benchmark' and input data is in '<path-to>/ga-benchmark/build' folder.
data = read_data(folder=input_path, verbose=False)

Reading JSON files:   6%|███████▎                                                                                                            | 5/79 [00:00<00:00, 9391.63it/s]

Grade
{'SystemName ': ' Linux ', ' SystemVersion ': ' 6.2.0-2-rt3-MANJARO ', ' CompilerID ': ' GNU ', ' CompilerVersion ': ' 13.1.1 ', ' Solution ': ' GATL ', ' Model ': ' 0x01 ', ' D ': 3, ' Grade ': 0, 'iterations': 10, 'repeats': 1}


KeyError: 'Grade'

The `SOLUTIONS` variable gives fancy names to the compared solutions. You have to include your solution here if you have added it to `ga-benchmark`.

In [5]:
SOLUTIONS = {
    'Gaalet': {'description': 'Gaalet'},
    'Gaalop': {'description': 'Gaalop'},
    'Garamon': {'description': 'Garamon'},
    'GATL': {'description': 'GATL'},
    'GluCatFramedMulti': {'description': 'GluCat (framed)'},
    'GluCatMatrixMulti': {'description': 'GluCat (matrix)'},
    'TbGAL': {'description': 'TbGAL'},
    'Versor': {'description': 'Versor'}
}

# Configuration

The `make_configuration_gui` function creates the user interface to select the solutions, models of geometry, and operations used while computing tables and charts. All options are selected by default.

The `filter_data` function filters `data`-like dictionaries using the options selected in the user interface.

In [6]:
def make_configuration_gui(given_data: dict) -> dict:
    given_solutions, given_models, given_operations = available_data(given_data)
    checkboxes = dict()
    for context_key, (context, _) in given_data.items():
        context_tab_titles = list()
        context_tab_children = list()
        # Tab 1: Solutions
        solutions_checkboxes = dict()
        for solution_key in given_solutions[context_key]:
            solutions_checkboxes[solution_key] = widgets.Checkbox(description=SOLUTIONS[solution_key]['description'], value=True, indent=False)
        context_tab_titles.append('1. Solutions')
        context_tab_children.append(widgets.VBox(children=[solution_checkbox for _, solution_checkbox in sorted(solutions_checkboxes.items())]))
        # Tab 2: Models of Geometry
        models_checkboxes = dict()
        models_items = list()
        for ind, (model_key, dimensions) in enumerate(given_models[context_key].items()):
            model_checkboxes = list()
            for d in sorted(dimensions):
                d_checkbox = widgets.Checkbox(description='%d-D' % d, value=True, indent=False)
                models_checkboxes.setdefault(model_key, dict())[d] = d_checkbox
                model_checkboxes.append(d_checkbox)
            models_items.append(widgets.Label(value=MODELS[model_key]['description']))
            models_items.append(widgets.VBox(children=model_checkboxes))
        context_tab_titles.append('2. Models')
        context_tab_children.append(widgets.GridBox(models_items, layout=widgets.Layout(grid_template_columns='100px auto')))
        # Tabs 3.*: Groups of Operations
        operations_checkboxes = dict()
        for group_ind, (group_key, subgroups) in enumerate(sorted(given_operations[context_key].items())):
            group_items = list()
            for subgroup_ind, (subgroup_key, operations) in enumerate(sorted(subgroups.items())):
                subgroup_checkboxes = list()
                for operation_key in sorted(operations):
                    operation_checkbox = widgets.Checkbox(description=OPERATIONS[group_key]['subgroups'][subgroup_key]['operations'][operation_key]['description'], value=True, indent=False)
                    operations_checkboxes.setdefault(group_key, dict()).setdefault(subgroup_key, dict())[operation_key] = operation_checkbox
                    subgroup_checkboxes.append(operation_checkbox)
                group_items.append(widgets.Label(value=OPERATIONS[group_key]['subgroups'][subgroup_key]['description']))
                group_items.append(widgets.VBox(children=subgroup_checkboxes))
            context_tab_titles.append('3.%d. %s' % (group_ind + 1, OPERATIONS[group_key]['description']))
            context_tab_children.append(widgets.GridBox(group_items, layout=widgets.Layout(grid_template_columns='150px auto')))
        # The context tab
        context_tab = widgets.Tab()
        context_tab.children = context_tab_children
        for ind, title in enumerate(context_tab_titles):
            context_tab.set_title(ind, title)
        display(widgets.HTML(value='<p style="line-height: 1.5"><b>Context:</b> %s</p>' % context_to_str(context)))
        display(context_tab)
        checkboxes[context_key] = {'solutions': solutions_checkboxes, 'models': models_checkboxes, 'operations': operations_checkboxes}
    return checkboxes

configuration_checkboxes = make_configuration_gui(data)

def filter_data(given_data: dict) -> dict:
    filtered_data = dict()
    for context_key, (context, models) in data.items():
        for model_key, dimensions in models.items():
            for d, groups in dimensions.items():
                if configuration_checkboxes[context_key]['models'][model_key][d].value:
                    for group_key, subgroups in groups.items():
                        for subgroup_key, operations in subgroups.items():
                            for operation_key, solutions in operations.items():
                                if configuration_checkboxes[context_key]['operations'][group_key][subgroup_key][operation_key].value:
                                    for solution_key, cases in solutions.items():
                                        if configuration_checkboxes[context_key]['solutions'][solution_key].value:
                                            filtered_data.setdefault(context_key, (context, dict()))[1].setdefault(model_key, dict()).setdefault(d, dict()).setdefault(group_key, dict()).setdefault(subgroup_key, dict()).setdefault(operation_key, dict())[solution_key] = cases
    return filtered_data

NameError: name 'data' is not defined

## Results

This section presents three types of results: the [table of features](#table_of_features), the [ranking](#ranking), and [performance charts](#performance_charts).

### Table of Features

The following table summarizes the models of geometry and the operations supported by the compared solution.

In [7]:
display(widgets.HTML(value=make_table_of_features(data=filter_data(data), solutions=SOLUTIONS), placeholder='Table of Features'))

HTML(value='<!DOCTYPE html><html><head><style>.verticalTableHeader {text-align: center; line-height: 1.0; whit…

### Ranking

The following tables classify the compared solutions using the gold first method, *i.e.,* based first on the number of gold medals, then silver, and so on. The medals are distributed among the solutions for testing cases implemented by all of them.

In [8]:
tables = make_ranking(data=filter_data(data), solutions=SOLUTIONS)
for table in tables:
    display(widgets.HTML(value=table, placeholder='Ranking'))

KeyError: 'Gaalet'

### Performance Charts

The following charts summarize the performance of the compared solution. Set the `result_path` variable to change the directory where the chart files will be written.

In [ ]:
result_path = path.join(path.abspath(getcwd()), 'results')  # Assuming that the working directory is '<path-to>/ga-benchmark'.
tables = make_performance_charts(data=filter_data(data), solutions=SOLUTIONS, folder=result_path)
for table in tables:
    display(widgets.HTML(value=table, placeholder='Performance Chart'))

Drawing performance charts:   0%|                                                                                                                     | 0/546 [00:00<?, ?it/s]/usr/lib/python3.11/site-packages/matplotlib/colors.py:1355: RuntimeWarning: overflow encountered in divide
  resdat /= (vmax - vmin)
/usr/lib/python3.11/site-packages/matplotlib/colors.py:718: RuntimeWarning: overflow encountered in multiply
  xa *= self.N
Drawing performance charts:  55%|███████████████████████████████████████████████████████████▏                                               | 302/546 [00:34<00:28,  8.66it/s]